In [57]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score,mean_squared_error

### Paths

In [58]:
#File paths
main_path_energy ='../data_base/paths_energy/'
main_path_total_energy = '../data_base/paths_totals/'
#main_path_turns = "/home/sebastian/Desktop/"

energy_path11_pd = pd.read_csv(main_path_energy + 'feat_pathv11.csv')
energy_path12_pd = pd.read_csv(main_path_energy + 'feat_pathv12-2.csv')
energy_path13_pd = pd.read_csv(main_path_energy + 'feat_pathv13.csv')

energy_path_list = [energy_path11_pd,energy_path12_pd,energy_path13_pd]
complete_energy_pd = pd.concat(energy_path_list)

total_path11_pd = pd.read_csv(main_path_total_energy + 'totalF_pathv11.csv')
total_path12_pd = pd.read_csv(main_path_total_energy + 'totalF_pathv12-2.csv')
total_path13_pd = pd.read_csv(main_path_total_energy + 'totalF_pathv13.csv')

total_path_list = [total_path11_pd,total_path12_pd,total_path13_pd]
complete_total_pd = pd.concat(total_path_list)

### Data split and preprocessing - complete_energy_pd - All data

this data comes from the folder "paths_energy"

In [59]:
X = complete_energy_pd.drop(columns=["missing_points","path_num","sim_drone_time","Energy","sim_point_dist","sim_Xdist","sim_Ydist","avg_error_target_dist","avg_energy"],axis=1)
y = complete_energy_pd["Energy"].to_numpy()
print(X.head())
X = X.to_numpy()

x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=35)

scaler = StandardScaler()

x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

print(f'Train size/Test size {len(x_train),len(x_test)}')

   teo_point_time  teo_point_dist  teo_Xdist  teo_Ydist  teo_point_vel  \
0             190       14.142136       10.0       10.0       0.074432   
1             233       12.727922        9.0        9.0       0.054626   
2             248       23.323808       20.0       12.0       0.094048   
3             157        8.000000        0.0        8.0       0.050955   
4             236       19.235384        3.0       19.0       0.081506   

   teo_Xvel  teo_Yvel  
0  0.052632  0.052632  
1  0.038627  0.038627  
2  0.080645  0.048387  
3  0.043694  0.026216  
4  0.012712  0.080508  
Train size/Test size (2486, 622)


#### Fit Linear regression

In [60]:
linearRe = LinearRegression()
model = linearRe.fit(x_train,y_train)

#### Accuracy

In [61]:
start_time = time.time()
predict = linearRe.predict(x_test)

r2 = r2_score(y_test,predict)
rmse = mean_squared_error(y_test,predict,squared=False)


print(f'r2 score: {r2}')
print(f'RMSE score: {rmse}')
print('Execution time = ' + str(time.time() - start_time))

r2 score: 0.6561379072357906
RMSE score: 1121879.6006989288
Execution time = 0.003353595733642578


In [62]:
linearRe.coef_

array([1366434.15339698,  290309.23359589,   76709.19878347,
         69054.8220471 ,  -97272.62798201,  -35888.14760811,
        -31577.84831178])

### Data split and preprocessing - complete_energy_pd - with eps

In [63]:
eps = 5
complete_energy_pd = complete_energy_pd[(complete_energy_pd['sim_point_dist']-complete_energy_pd['teo_point_dist']) < eps]
print(f'new shape of complete_energy_pd = {complete_energy_pd.shape}')

new shape of complete_energy_pd = (2577, 16)


In [64]:
X = complete_energy_pd.drop(columns=["missing_points","path_num","sim_drone_time","Energy","sim_point_dist","sim_Xdist","sim_Ydist","avg_error_target_dist","avg_energy"],axis=1)
y = complete_energy_pd["Energy"].to_numpy()

X = X.to_numpy()

x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=35)

scaler2 = StandardScaler()

x_train = scaler2.fit_transform(x_train)
x_test = scaler2.transform(x_test)

print(f'Train size/Test size {len(x_train),len(x_test)}')

Train size/Test size (2061, 516)


#### Fit Linear regression

In [65]:
linearRe2 = LinearRegression()
model = linearRe2.fit(x_train,y_train)

#### Accuracy

In [66]:
start_time = time.time()
predict = linearRe2.predict(x_test)

r2 = r2_score(y_test,predict)
rmse = mean_squared_error(y_test,predict,squared=False)


print(f'r2 score: {r2}')
print(f'RMSE score: {rmse}')
print('Execution time = ' + str(time.time() - start_time))

r2 score: 0.6940573597433006
RMSE score: 791884.1676508978
Execution time = 0.0026242733001708984


### Data split and preprocessing - complete_total_pd

this data comes from "path_totals" folder

In [67]:
X = complete_total_pd.drop(columns=["path_num","energy"],axis=1)
y = complete_total_pd["energy"].to_numpy()

X = X.to_numpy()

x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=35)

scaler = StandardScaler()

x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

print(f'Train size/Test size {len(x_train),len(x_test)}')

Train size/Test size (466, 117)


#### Fit Linear regression

In [68]:
linearRe3 = LinearRegression()
model = linearRe3.fit(x_train,y_train)

#### Accuracy

In [69]:
start_time = time.time()
predict = linearRe3.predict(x_test)

r2 = r2_score(y_test,predict)
rmse = mean_squared_error(y_test,predict,squared=False)


print(f'r2 score: {r2}')
print(f'RMSE score: {rmse}')
print('Execution time = ' + str(time.time() - start_time))

r2 score: -0.05646625577785702
RMSE score: 5357820.774397302
Execution time = 0.004519462585449219
